In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import re
import pyLDAvis.gensim_models
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
import json
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import itertools
import collections

C:\Users\User\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\User\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\User\anaconda3\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
C:\Users\User\anaconda3\lib\site-packages\setuptools\_distutils\version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


### LDA 참고: https://wikidocs.net/30708

In [19]:
df= pd.read_csv('df_title_완.csv',encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206966 entries, 0 to 206965
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   206966 non-null  object
 1   date    206966 non-null  object
dtypes: object(2)
memory usage: 3.2+ MB


In [25]:
df['title'][0:4]

0        \n광룡 하스미\n[1] 
1    \n머리는왜자라는것인\n[24] 
2      \n일케생긴 여갤러 누구임\n
3          \n배고프다\n[2] 
Name: title, dtype: object

In [20]:
#광고, 설문 데이터 제거
values_to_remove = df['title'][0:3]
df = df[~df['title'].isin(values_to_remove)]
# index 초기화
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195212 entries, 0 to 195211
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   195212 non-null  object
 1   date    195212 non-null  object
dtypes: object(2)
memory usage: 3.0+ MB


In [21]:
def remove_non(text):
    # 정규 표현식을 사용하여 한글, 숫자, 알파벳과 띄어쓰기만 남김
    pattern = r'[^\w가-힣\s]+'  # 한글, 숫자, 알파벳, 띄어쓰기를 제외한 모든 문자 제거
    return re.sub(pattern, '', text)

In [22]:
df['clean_doc'] = df['title'].apply(remove_non)
# 길이가 1이하인 단어는 제거 (길이가 짧은 단어 제거)
df['clean_doc'] = df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
# # 전체 단어에 대한 소문자 변환
df['clean_doc'] = df['clean_doc'].apply(lambda x: x.lower())
#숫자제거
df['clean_doc'] = df['clean_doc'].apply(lambda x: re.sub(r'\d', '', x))

In [23]:
df.to_csv('던파_간단히전처리.csv',encoding='utf-8')

In [7]:
#불용어 제거를 위해 토큰화 작업을 수행하였지만, TfidfVectorizer(TF-IDF 실습 참고)는
#기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다.
#그렇기 때문에 TfidfVectorizer를 사용해서 TF-IDF 행렬을 만들기 위해서
#다시 토큰화 작업을 역으로 취소하는 작업을 수행해보도록 하겠습니다. 이를 역토큰화(Detokenization)라고 합니다.
detokenized_doc = []
for i in range(len(df)):
    
    t = ' '.join(tokenized_doc[i])
    
    detokenized_doc.append(t)
    

df['clean_doc'] = detokenized_doc
df['clean_doc']

0                   광룡 하스미
1                머리는왜자라는것인
2             일케생긴 여갤러 누구임
3                     배고프다
4               던붕이 스펙 ㅁㅌㅊ
                ...       
195207          워터밤 오라 별로임
195208       지마갤 유동행님들 거지임
195209           노래듣고자기 궈궈
195210    빨리 월요일이 왔으면 좋겠어요
195211          짱깨룩 못벗는 이유
Name: clean_doc, Length: 195212, dtype: object

In [8]:
df=df[df['date']!='10.14']
df['date'] = '2023.' + df['date']
df['date'] = pd.to_datetime(df['date'], format='%Y.%m.%d')
df['month']=df['date'].dt.month

df.info()
# 날짜 범위 설정
start_date_1 = '2023-07-13'
end_date_1 = '2023-08-12'
start_date_2 = '2023-08-13'
end_date_2 = '2023-09-12'
start_date_3 = '2023-09-13'
end_date_3 = '2023-10-12'

# 날짜 범위에 따라 데이터프레임 분할
df_1 = df[(df['date'] >= start_date_1) & (df['date'] <= end_date_1)]
df_2 = df[(df['date'] >= start_date_2) & (df['date'] <= end_date_2)]
df_3 = df[(df['date'] >= start_date_3) & (df['date'] <= end_date_3)]
df = df.reset_index(drop=True)
df_1 = df.reset_index(drop=True)
df_2 = df.reset_index(drop=True)
df_3 = df.reset_index(drop=True)


<class 'pandas.core.frame.DataFrame'>
Index: 195168 entries, 44 to 195211
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   title      195168 non-null  object        
 1   date       195168 non-null  datetime64[ns]
 2   clean_doc  195168 non-null  object        
 3   month      195168 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 6.7+ MB


In [9]:
vectorizer = TfidfVectorizer(stop_words='english',max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(df['clean_doc'])
X_1 = vectorizer.fit_transform(df_1['clean_doc'])
X_2 = vectorizer.fit_transform(df_2['clean_doc'])
X_3 = vectorizer.fit_transform(df_3['clean_doc'])
# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (195168, 250171)


In [10]:
#토픽 개수
topic_num =10

In [11]:
# n_components : n개의 토픽을 가졌다고 가정
svd_model = TruncatedSVD(n_components=topic_num, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_) 

10

In [12]:
np.shape(svd_model.components_)

(10, 250171)

In [13]:
terms = vectorizer.get_feature_names_out() # 단어 집합.

def get_topics(components, feature_names, n=topic_num):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('여거너랑', 1.0), ('사랑나누기', 0.001), ('하고싶네', 0.0009), ('여프리하고', 0.00088), ('쓰리섬하고싶다', 0.00085), ('한장', 0.0), ('퇴근', 0.0), ('여갤루랑', 0.0), ('싶네', 0.0), ('검신', 0.0)]
Topic 2: [('이벤트', 0.67023), ('피시방', 0.45068), ('대리', 0.41498), ('pc방', 0.34408), ('모집합니다', 0.1314), ('시간당원', 0.12999), ('새로운', 0.1293), ('뉴비', 0.04473), ('참여', 0.02313), ('질문', 0.01831)]
Topic 3: [('길드는', 0.65126), ('서버는', 0.47551), ('통합', 0.43608), ('던갤러', 0.31894), ('보라타운', 0.1784), ('나만운없어', 0.14308), ('고소', 0.05184), ('리부트', 0.03265), ('갤길드는', 0.01809), ('이쁨왕피롱님', 0.01745)]
Topic 4: [('몸매', 0.99958), ('적당한', 0.01437), ('초전도체', 0.00828), ('우왓', 0.00716), ('호불호', 0.00694), ('던순이', 0.00655), ('ㄷㄷ', 0.00629), ('갈리는', 0.00588), ('최고의', 0.00527), ('여갤러', 0.00429)]
Topic 5: [('반갑상어', 1.0), ('aaa', 0.0), ('뉴비', 0.0), ('던갤', 0.0), ('질문', 0.0), ('메이플', 0.0), ('잘자요', 0.0), ('질문좀', 0.0), ('좋은아침', 0.0), ('쿠폰', 0.0)]
Topic 6: [('뉴비', 0.93789), ('질문', 0.25915), ('질문좀', 0.1155), ('던갤', 0.07162), ('질문있어요', 0.06807), ('커스텀', 0.05345

In [14]:
# n_components : n개의 토픽을 가졌다고 가정
svd_model = TruncatedSVD(n_components=topic_num, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

10

In [15]:
terms = vectorizer.get_feature_names_out() # 단어 집합.

def get_topics(components, feature_names, n=topic_num):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('여거너랑', 1.0), ('사랑나누기', 0.001), ('하고싶네', 0.0009), ('여프리하고', 0.00088), ('쓰리섬하고싶다', 0.00085), ('한장', 0.0), ('퇴근', 0.0), ('여갤루랑', 0.0), ('싶네', 0.0), ('검신', 0.0)]
Topic 2: [('이벤트', 0.67023), ('피시방', 0.45068), ('대리', 0.41498), ('pc방', 0.34408), ('모집합니다', 0.1314), ('시간당원', 0.12999), ('새로운', 0.1293), ('뉴비', 0.04473), ('참여', 0.02313), ('질문', 0.01831)]
Topic 3: [('길드는', 0.65126), ('서버는', 0.47551), ('통합', 0.43608), ('던갤러', 0.31894), ('보라타운', 0.1784), ('나만운없어', 0.14308), ('고소', 0.05184), ('리부트', 0.03265), ('갤길드는', 0.01809), ('이쁨왕피롱님', 0.01745)]
Topic 4: [('몸매', 0.99958), ('적당한', 0.01437), ('초전도체', 0.00828), ('우왓', 0.00716), ('호불호', 0.00694), ('던순이', 0.00655), ('ㄷㄷ', 0.00629), ('갈리는', 0.00588), ('최고의', 0.00527), ('여갤러', 0.00429)]
Topic 5: [('반갑상어', 1.0), ('aaa', 0.0), ('뉴비', 0.0), ('던갤', 0.0), ('질문', 0.0), ('메이플', 0.0), ('잘자요', 0.0), ('질문좀', 0.0), ('좋은아침', 0.0), ('쿠폰', 0.0)]
Topic 6: [('뉴비', 0.93789), ('질문', 0.25915), ('질문좀', 0.1155), ('던갤', 0.07162), ('질문있어요', 0.06807), ('커스텀', 0.05345

In [16]:
documents = [doc.split() for doc in df['clean_doc']]
dictionary = corpora.Dictionary(documents)


corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

## 2) LDA 모델 훈련시키기

In [17]:
import gensim
NUM_TOPICS = 5 # 5개의 토픽, k=5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.007*"던담" + 0.006*"바칼" + 0.006*"던갤" + 0.004*"커스텀"')
(1, '0.007*"엘마" + 0.004*"명성" + 0.003*"휘장" + 0.003*"하나"')
(2, '0.009*"이벤트" + 0.007*"vs" + 0.005*"하칼" + 0.004*"피시방"')
(3, '0.008*"뉴비" + 0.005*"상향" + 0.003*"게이볼그" + 0.003*"회랑"')
(4, '0.006*"배메" + 0.006*"메이플" + 0.006*"캐릭" + 0.005*"버퍼"')


In [18]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)